# Начало начал
качаем датасет. возьмем для работы датасет рика и морти с платформы кагл.

In [1]:
import pandas as pd
all_rick = pd.read_csv('./data/RickAndMortyScripts.csv')
all_rick.head(5)

,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."


In [2]:
# Собираем контектст ответа
contexted = []

n = 7

for i in range(n, len(all_rick['line'])):
  row = []
  prev = i - 1 - n
  for j in range(i, prev, -1):
    row.append(all_rick['line'][j])
  contexted.append(row)

In [3]:
len(contexted)

1898

In [4]:
# формируем названия колонок контекста
columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4',
 'context/5']

In [5]:
# собираем датасет
knowledge_df = pd.DataFrame.from_records(contexted, columns=columns)
knowledge_df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,"What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go...",Ow! Ow! You're tugging me too hard!,"Come on, I got a surprise for you. Come on, h...",It's the middle of the night. What are you tal...,"I got a surprise for you, Morty.","What, Rick? What’s going on?",Morty! You gotta come on. Jus'... you gotta co...
1,"Yeah, Rick... I-it's great. Is this the surprise?","What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go...",Ow! Ow! You're tugging me too hard!,"Come on, I got a surprise for you. Come on, h...",It's the middle of the night. What are you tal...,"I got a surprise for you, Morty.","What, Rick? What’s going on?"
2,Morty. I had to... I had to do it. I had— I ha...,"Yeah, Rick... I-it's great. Is this the surprise?","What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go...",Ow! Ow! You're tugging me too hard!,"Come on, I got a surprise for you. Come on, h...",It's the middle of the night. What are you tal...,"I got a surprise for you, Morty."
3,What?! A bomb?!,Morty. I had to... I had to do it. I had— I ha...,"Yeah, Rick... I-it's great. Is this the surprise?","What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go...",Ow! Ow! You're tugging me too hard!,"Come on, I got a surprise for you. Come on, h...",It's the middle of the night. What are you tal...
4,We're gonna drop it down there just get a whol...,What?! A bomb?!,Morty. I had to... I had to do it. I had— I ha...,"Yeah, Rick... I-it's great. Is this the surprise?","What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go...",Ow! Ow! You're tugging me too hard!,"Come on, I got a surprise for you. Come on, h..."


In [6]:
knowledge_df.to_csv("knowledge_df.csv", index=False, encoding="utf-8")

# попробуем векторизовать контекст полученного датасета

In [7]:
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRReader
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

/Users/paru4ik/miniconda3/envs/chatbot/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Using device: cpu


In [8]:
# Загрузка предварительно обученного биэнкодера DPR и токенизатора
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base").to(device)
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

# Загрузка предварительно обученного ридера DPR
reader = DPRReader.from_pretrained("facebook/dpr-reader-single-nq-base").to(device)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenize

In [9]:
# Функция предобработки текста
def tokenize_and_vectorize(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.pooler_output[0].cpu().numpy()

In [10]:
context_vectors = []
for col in knowledge_df.columns[1:]:
    for context in tqdm(knowledge_df[col], desc=f"Vectorizing {col}"):
        context_vectors.append(tokenize_and_vectorize(context, context_encoder, tokenizer))

Vectorizing context/5: 100%|████████████████| 1898/1898 [00:59<00:00, 31.99it/s]


In [11]:
import numpy as np
# Сохранение векторов контекстов в файл на будущее
np.save("context_vectors.npy", context_vectors)

In [12]:
# Функция поиска наиболее подходящего ответа
def get_response(query, context_vectors, knowledge_df, model, tokenizer):
    query_vector = tokenize_and_vectorize(query, model, tokenizer)
    similarity_scores = cosine_similarity([query_vector], context_vectors)
    most_similar_index = similarity_scores.argmax()
    col_index = most_similar_index % len(knowledge_df.columns[1:])
    row_index = most_similar_index // len(knowledge_df.columns[1:])
    return knowledge_df.iloc[row_index, 0]

In [13]:
user_input = input("User: ")
while user_input.lower() != 'пока':
    response = get_response(user_input, context_vectors, knowledge_df, context_encoder, tokenizer)
    print("Bot:", response)
    user_input = input("User: ")

User:  Hey


Bot: You ask a lot of questions, Morty. Not very charismatic. It makes you kind of an  under-  underfoot figure.


User:  sorry


Bot: Don't worry about it, Morty. Just come help me get these seeds, all right, buddy?


User:  Yes, its all right. So, what we will do today?


Bot: Uh, yeah, maybe I am. I'm my father's daughter. I'm smart. Why do you think I'm a heart surgeon?


User:  I dont thisk so


Bot: It's personal.


User:  пока
